# Employee report

In [3]:
import random
import ipywidgets as widgets
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)


import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(colorscale='plotly', world_readable=True)

# Extra options
pd.options.display.max_rows = 30
pd.options.display.max_columns = 25

from datetime import date
import src.compute.changelogs as changelogs
from ipywidgets import interact
from src.db.utils import SnowflakeWrapper
from src.compute.utils import Interval, get_statuses
from src.compute.developer import get_developers, get_developer_ids
from src.compute.tickets import get_ticket_priorities, get_ticket_types

conn = SnowflakeWrapper.create_snowflake_connection()
sw = SnowflakeWrapper(conn)

dev_ids = get_developer_ids(sw)
ticket_statuses = {val:val for val in get_statuses()}
ticket_statuses["All"] = None
priorities = {val:val for val in get_ticket_priorities(sw)}
priorities["All"] = None
types = {val:val for val in get_ticket_types(sw)}
types["All"] = None

In [4]:
label_map = {
    "metrics": {
        "Average days": "AVG_DAYS",
        "Degree of cycling": "DegreeOfCycling",
    },
    "dims": {
        "type":  'IssueType',
        "priority": 'IssuePriority',
        "status": "MERGED_STATUS",
    }
}

def plot_distribution_counts(dev_id, status, x_col, ipriority, itype, date_from, date_to, max_days):
    current_interval = Interval(date_from, date_to)
    plot_df = get_developers(
        sw,
        current_interval,
        user_filters={
            "status": ticket_statuses[status],
            "priority": priorities[ipriority],
            "type": types[itype],
        },
        max_duration=max_days
    )
    if plot_df.empty:
        print("No data for chosen combination.")
        return
    chosen_dev = plot_df.loc[plot_df["ASSIGNEE"] == dev_id]
    metric = label_map["metrics"][x_col]
    max_val, min_val  = plot_df[metric].max(), plot_df[metric].min()
    bins = int(plot_df[label_map["metrics"][x_col]].max() - plot_df[label_map["metrics"][x_col]].min()) + 1
    fig = px.histogram(
        plot_df,
        x=label_map["metrics"][x_col],
        title=f"There are {len(plot_df.index)} active developers on the interval {current_interval.pretty_str()}",
        hover_data=plot_df.columns,
        nbins=bins,
        labels = {
            metric: x_col,
        }
    )
    if not chosen_dev.empty:
        mean_val = chosen_dev[metric].iloc[0]
        fig.update_layout(shapes=[
            dict(
                type='line',
                yref='paper', y0= 0, y1= 1,
                xref='x', x0= mean_val, x1= mean_val,
                line=dict(
                    color="Red",
                    width=4,
                    dash="dashdot",
                )
            )
        ])
    fig.update_layout(
        xaxis = dict(
            range=[min_val - 1, max_val + 1],  # sets the range of xaxis
            tickmode='linear'
        ),
        bargap=0.01
    )
    return fig

_ = interact(
        plot_distribution_counts,
        dev_id=widgets.widgets.Text(value='marko.prelevikj', placeholder='Developer ID', description='Choose Developer', disabled=False),
        x_col=widgets.Dropdown(options=label_map["metrics"].keys(), description="Metric"),
        status=widgets.Dropdown(options=sorted(ticket_statuses.keys()), description="Ticket status"),
        ipriority=widgets.Dropdown(options=sorted(priorities.keys()), description="Ticket priority"),
        itype=widgets.Dropdown(options=sorted(types.keys()), description="Ticket type"),
        date_from=widgets.DatePicker(value=pd.to_datetime('2019-10-01'), description="Date from"),
        date_to=widgets.DatePicker(value=pd.to_datetime('2020-01-01'), description="Date to"),
        max_days=widgets.IntSlider(value=10, min=1, max=30, description="Max days to resolve")
    )

interactive(children=(Text(value='marko.prelevikj', description='Choose Developer', placeholder='Developer ID'…